### Pré-processamento

In [259]:
# Importando libs para análise
import numpy as np
import pandas as pd
import re

In [260]:
# Obtendo dados
df = pd.read_csv('dataset.csv')
df.columns = df.columns.str.lower()
df.head()

,title,content
0,"Ato Declaratorio Interpretativo SRF nº 15, de ...","Ato Declaratório Interpretativo SRF nº 15, de ..."
1,"Ato Declaratorio nº 01, de 2013",* Publicado no DOE em 01/02/2013 ATO DECLARA...
2,"Ato Declaratorio nº 02, de 2013",* Publicado no DOE em 01/02/2013 ATO DECLARA...
3,"Ato Declaratório nº 01, de 1998",ATO DECLARATÓRIO Nº 01/1998 07/01/1998 * Publi...
4,"Ato Declaratório nº 01, de 1999",ATO DECLARATÓRIO Nº 01/1999 14/01/1999 * Publi...


In [261]:
# Tamanho do dataset
df.shape[0]

3606

### Verificando por dados faltantes

In [262]:
# Verificando por dados faltantes
df.isnull().sum()

title      0
content    2
dtype: int64

In [263]:
# Removendo dados faltantes
df.dropna(inplace=True)
# Novo tamanho do dataset
df.shape[0]

3604

### Verificando por ambiguidades

In [264]:
tipos_de_documentos = ["ato declaratório", "ato declaratorio", "decreto", "instrução normativa", 
                      "lei", "lei complementar", "norma de execução", "nota explicativa"]

In [266]:
df['title'].head()

0    Ato Declaratorio Interpretativo SRF nº 15, de ...
1                      Ato Declaratorio nº 01, de 2013
2                      Ato Declaratorio nº 02, de 2013
3                      Ato Declaratório nº 01, de 1998
4                      Ato Declaratório nº 01, de 1999
Name: title, dtype: object

In [267]:
def num_ocorrencias(documento):
    """Conta o número de ocorrências para cada tipo de classe de documento"""
    return df['title'].apply(lambda t: bool(re.match(documento, t.lower()))).sum()

def indices_ocorrencias(documento):
    """Gera uma lista de índices para cada ocorrência de tipo de classe de documento"""
    return df.index[df['title'].apply(lambda t: bool(re.match(documento, t.lower())))].tolist()


dist_documentos = [num_ocorrencias(documento) for documento in tipos_de_documentos]
indices_documentos = [indices_ocorrencias(documento) for documento in tipos_de_documentos]

In [268]:
indice = 0

print("Distribuição das classes:\n")

for documento, qtd in zip(tipos_de_documentos, dist_documentos):
    print(indice, documento, qtd)
    indice += 1

print("\nTotal:", sum(dist_documentos))

Distribuição das classes:

0 ato declaratório 669
1 ato declaratorio 3
2 decreto 1024
3 instrução normativa 1376
4 lei 331
5 lei complementar 72
6 norma de execução 117
7 nota explicativa 84

Total: 3676


Há alguma diferença no número de exemplares de classes total e o número total de dados?

In [269]:
df.shape[0], sum(dist_documentos)

(3604, 3676)

In [270]:
conjs_indices_documentos = [set(lista) for lista in indices_documentos]

# Observando se há interseções entre os índices dois a dois (i.e. se há dados com duas classes)
for i in range(0, 8):
    for j in range(0, 8):
        if i != j and set.intersection(conjs_indices_documentos[i], conjs_indices_documentos[j]):
            print(f"i: {i} j: {j}")

i: 4 j: 5
i: 5 j: 4


In [271]:
intersect_lista = [el for el in set.intersection(conjs_indices_documentos[4], conjs_indices_documentos[5])]
df['title'].iloc[df.index[intersect_lista]]

3077    Lei Complementar 0024, de 1975 (Cópia em Traba...
3078    Lei Complementar 0024, de 1975 (Cópia em Traba...
3079    Lei Complementar 0024, de 1975 (Cópia em Traba...
3080    Lei Complementar 0024, de 1975 (Cópia em Traba...
3081    Lei Complementar 0024, de 1975 (Cópia em Traba...
                              ...                        
3144                      Lei Complementar nº 50, de 2004
3145                      Lei Complementar nº 81, de 2009
3146                      Lei Complementar nº 81, de 2009
3147                               Lei n° 08.137, de 1990
3148                               Lei n° 08.397, de 1992
Name: title, Length: 72, dtype: object

Os dados supostamente ambíguos referem-se a títulos com a palavra "lei", mas definimos duas classes diferentes, i.e., "lei" e "lei complementar". Uma vez que a palavra *lei* é encontrada em ambos, não há problemas. Como a classe é contada duas vezes, retiremos a diferença:

In [272]:
sum(dist_documentos) - len(intersect_lista), df.shape[0] # (Diferença, Tamanho real do dataset)

(3604, 3604)

Aparentemente, não há ambiguidade nos títulos. Podemos separar os dados nas classes que definimos.

In [273]:
df.head()

,title,content
0,"Ato Declaratorio Interpretativo SRF nº 15, de ...","Ato Declaratório Interpretativo SRF nº 15, de ..."
1,"Ato Declaratorio nº 01, de 2013",* Publicado no DOE em 01/02/2013 ATO DECLARA...
2,"Ato Declaratorio nº 02, de 2013",* Publicado no DOE em 01/02/2013 ATO DECLARA...
3,"Ato Declaratório nº 01, de 1998",ATO DECLARATÓRIO Nº 01/1998 07/01/1998 * Publi...
4,"Ato Declaratório nº 01, de 1999",ATO DECLARATÓRIO Nº 01/1999 14/01/1999 * Publi...
